### Bibliotecas e información de la sesión

In [1]:
# Cargamos las librerías
library(tidyverse)
library(ggplot2)
library(caret)
library(RWeka)
library(AUC)
library(VIM)
library(missForest)
library(mice)
library(klaR)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.5     v dplyr   1.0.7
v tidyr   1.1.4     v stringr 1.4.0
v readr   2.0.2     v forcats 0.5.1

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: 'caret'


The following object is masked from 'package:purrr':

    lift


AUC 0.3.0

Type AUCNews() to see the change log and ?AUC to get an overview.


Attaching package: 'AUC'


The following objects are masked from 'package:caret':

    sensitivity, specificity


Loading required package: colorspace

Loading required package: grid

VIM is ready to use.


Suggestions and bug-reports can be submitted at: https://github.com/statistikat/VIM/issues


Attaching package: 'VIM'


In [2]:
# Se informa de las características y el entorno en el que se ha desarrolado el trabajo
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 10 x64 (build 19043)

Matrix products: default

locale:
[1] LC_COLLATE=Spanish_Spain.1252  LC_CTYPE=Spanish_Spain.1252   
[3] LC_MONETARY=Spanish_Spain.1252 LC_NUMERIC=C                  
[5] LC_TIME=Spanish_Spain.1252    

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] klaR_0.6-15         MASS_7.3-54         mice_3.13.0        
 [4] missForest_1.4      itertools_0.1-3     iterators_1.0.13   
 [7] foreach_1.5.1       randomForest_4.6-14 VIM_6.1.1          
[10] colorspace_2.0-2    AUC_0.3.0           RWeka_0.4-43       
[13] caret_6.0-90        lattice_0.20-45     forcats_0.5.1      
[16] stringr_1.4.0       dplyr_1.0.7         purrr_0.3.4        
[19] readr_2.0.2         tidyr_1.1.4         tibble_3.1.5       
[22] ggplot2_3.3.5       tidyverse_1.3.1    

loaded via a namespace (and not attached):


## Cargar los datos

In [3]:
# Cargar datos de entrenamiento
# Imputación por MICE
tra_feat_noNAmice = read.csv('X_train_noNAmice.csv', header=TRUE, sep=',',
                    na.strings = c('?','', 'NA'))
# Imputación por knn
tra_feat_noNAknn = read.csv('train_features_knn_imput.csv', header=TRUE, sep=',',
                    na.strings = c('?','', 'NA'))
# Imputación por miss Forest
tra_feat_noNAmissF = read.csv('train_features_missF_imput.csv', header=TRUE, sep=',',
                    na.strings = c('?','', 'NA'))
# Datos originales
tra_feat_original = read.csv('training_set_features.csv', header=TRUE, sep=',',
                    na.strings = c('?','', 'NA'))
# Cargar labels de entrenamiento
tra_lab = read.csv('training_set_labels.csv', header=TRUE, sep=',',
                    na.strings = c('?','', 'NA'))
# Cargar datos de entrenamiento
# Imputación por MICE
test_feat_noNAmice = read.csv('X_test_noNAmice.csv', header=TRUE, sep=',',
                    na.strings = c('?','', 'NA'))
# Imputación por knn
test_feat_noNAknn = read.csv('test_features_knn_imput.csv', header=TRUE, sep=',',
                    na.strings = c('?','', 'NA'))
# Imputación por miss Forest
test_feat_noNAmissF = read.csv('test_features_missF_imput.csv', header=TRUE, sep=',',
                    na.strings = c('?','', 'NA'))
# Datos originales
test_feat_original = read.csv('test_set_features.csv', header=TRUE, sep=',',
                    na.strings = c('?','', 'NA'))
# Cargar el modelo para las entregas
submission = read.csv('submission_format.csv', header=TRUE, sep=',')

In [4]:
# Convertimos las variables de los dataframe a factor
tra_feat_noNAmice = as.data.frame((lapply(tra_feat_noNAmice[, -1], as.factor)))
tra_feat_noNAknn = as.data.frame((lapply(tra_feat_noNAknn[, -1], as.factor)))
tra_feat_noNAmissF = as.data.frame((lapply(tra_feat_noNAmissF[, -1], as.factor)))
tra_feat_original = as.data.frame((lapply(tra_feat_original[, -1], as.factor)))
tra_lab = as.data.frame(lapply(tra_lab[, -1], as.factor))
test_feat_noNAmice = as.data.frame((lapply(test_feat_noNAmice[, -1], as.factor)))
test_feat_noNAknn = as.data.frame((lapply(test_feat_noNAknn[, -1], as.factor)))
test_feat_noNAmissF = as.data.frame((lapply(test_feat_noNAmissF[, -1], as.factor)))
test_feat_original = as.data.frame((lapply(test_feat_original[, -1], as.factor)))

## Ensembles

### Forest

#### Forest CV

Se crean divisiones del data set de entrenamiento que se usarán para entrenar y evaluar el algoritmo

In [7]:
# Se crean subsets de los datos para entrenamiento y evaluación
shuffle_ds = sample(dim(tra_feat_noNAmice)[1])
pct90 = (dim(tra_feat_noNAmice)[1] * 90) %/% 100
tra_feat_sam = tra_feat_noNAmice[shuffle_ds[1:pct90], ]
tra_lab_sam = tra_lab[shuffle_ds[1:pct90], ]
tst_feat_sam = tra_feat_noNAmice[shuffle_ds[(pct90+1):dim(tra_feat_noNAmice)[1]], ]
tst_lab_sam = tra_lab[shuffle_ds[(pct90+1):dim(tra_lab)[1]], ]

In [8]:
# Se crea una función que se usa para testear el rendimiento del forest ensemble
accuracy_of_ensemble_h1n1 <- function(method, metric, tst_feat_sam, tra_feat_sam, ctrl, grid, b = 100) {
    # Se crea la variable en la que se guardarán los datos
    Pred_h1n1_prob = data.frame(X0 = rep(0.0, dim(tst_feat_sam)[1]), X1 = rep(0.0, dim(tst_feat_sam)[1]))
    # Se calcula el 5 porciento del número de muestras
    pct05 = (dim(tra_feat_sam)[1] * 5) %/% 100
    # Para cada división del dataset
    for (i in 1:b){
        # Se toma una muestra aleatoria
        shuffle_ds = sample(dim(tra_feat_sam)[1])
        ind_feat = sample(length(colnames(tra_feat_sam)), 6)
        # Se entrena un clasificador con esta muestra
        Fit_h1n1_i = train(tra_feat_sam[shuffle_ds[1:pct05], ind_feat], make.names(tra_lab_sam[shuffle_ds[1:pct05], 1]),
                      method = method,
                      metric = "ROC",
                      trControl = ctrl,
                      tuneGrid = grid)
        # Se predice con el clasificador la probabilidad de las muestras de test correspondiente a esta división
        Pred_h1n1_prob_i = predict(Fit_h1n1_i, newdata = tst_feat_sam, type = "prob")
        # Se añade la probabilidad ponderada
        Pred_h1n1_prob = Pred_h1n1_prob + (Pred_h1n1_prob_i / b)
    }
    # Se muestra y devuelve el rendimiento
    cat("AUC:", auc(roc(Pred_h1n1_prob$X1, tst_lab_sam$h1n1_vaccine)))
    return(auc(roc(Pred_h1n1_prob$X1, tst_lab_sam$h1n1_vaccine))) 
}

In [9]:
# Se crea el control del entrenamiento y los parámetros del clasificador
ctrl <- trainControl(method="cv", number=5, classProbs= TRUE, summaryFunction = twoClassSummary)
grid <- expand.grid(.fL=c(0), .usekernel=c(FALSE),.adjust=1)

In [10]:
options(warn = -1)
# Se comprueba el rendimiento del clasificador en los datos de entrenamiento para comprobar el funcionamiento correcto
accuracy_of_ensemble_h1n1(method = "nb", metric = "ROC", tst_feat_sam, tra_feat_sam, ctrl, grid)
options(warn = 0)

AUC: 0.7882392

[1] 0.7882392

#### Forest test

In [11]:
# Se crea una función que se usa para calcular la probabilidad de la salida h1n1
predict_with_ensemble_h1n1 <- function(method, metric, test_features, tra_feat, tra_lab, ctrl, grid, b = 100) {
    # Se crea la variable en la que se guardarán los datos
    Pred_h1n1_prob = data.frame(X0 = rep(0.0, dim(test_features)[1]), X1 = rep(0.0, dim(test_features)[1]))
    # Se calcula el 5 porciento del número de muestras
    pct05 = (dim(tra_feat)[1] * 5) %/% 100
    # Para cada división del dataset
    for (i in 1:b){
        # Se toma una muestra aleatoria
        shuffle_ds = sample(dim(tra_feat)[1])
        ind_feat = sample(length(colnames(tra_feat)), 6)
        # Se entrena un clasificador con esta muestra
        Fit_h1n1_i = train(tra_feat[shuffle_ds[1:pct05], ind_feat], make.names(tra_lab[shuffle_ds[1:pct05], 1]),
                        method = method,
                        metric = metric,
                        trControl = ctrl)

        # Se predice con el clasificador la probabilidad de las muestras de test correspondiente a esta división
        Pred_h1n1_prob_i = predict(Fit_h1n1_i, newdata = test_features, type = "prob")
        # Se añade la probabilidad ponderada
        Pred_h1n1_prob = Pred_h1n1_prob + (Pred_h1n1_prob_i / b)
        }
    # Se devuelve la probabilidad calculada
    return(Pred_h1n1_prob$X1)
}

In [12]:
# Se crea una función que se usa para calcular la probabilidad de la salida seasonal
predict_with_ensemble_seas <- function(method, metric, test_features, tra_feat, tra_lab, ctrl, grid, b = 100) {
    # Se crea la variable en la que se guardarán los datos
    Pred_seas_prob = data.frame(X0 = rep(0.0, dim(test_features)[1]), X1 = rep(0.0, dim(test_features)[1]))
    # Se calcula el 5 porciento del número de muestras
    pct05 = (dim(tra_feat)[1] * 5) %/% 100
    # Para cada división del dataset
    for (i in 1:b){
        # Se toma una muestra aleatoria
        shuffle_ds = sample(dim(tra_feat)[1])
        ind_feat = sample(length(colnames(tra_feat)), 6)        
        
        # Se entrena un clasificador con esta muestra
        Fit_seas_i = train(tra_feat[shuffle_ds[1:pct05], ind_feat], make.names(tra_lab[shuffle_ds[1:pct05], 2]),
                  method = method,
                  metric = metric,
                  trControl = ctrl)
  
        # Se predice con el clasificador la probabilidad de las muestras de test correspondiente a esta división
        Pred_seas_prob_i = predict(Fit_seas_i, newdata = test_features, type = "prob")
        # Se añade la probabilidad ponderada
        Pred_seas_prob = Pred_seas_prob + (Pred_seas_prob_i / b)
        }
    # Se devuelve la probabilidad calculada
    return(Pred_seas_prob$X1)
}

In [13]:
options(warn = -1)
# Se calcula la probabilidad dada por el algoritmo con los datos imputados con mice para ambas salidas
submission$h1n1_vaccine = predict_with_ensemble_h1n1("nb", "ROC", test_feat_noNAmice, tra_feat_noNAmice, tra_lab, ctrl, grid)
submission$seasonal_vaccine = predict_with_ensemble_seas("nb", "ROC", test_feat_noNAmice, tra_feat_noNAmice, tra_lab, ctrl, grid)
options(warn = 0)

In [14]:
# Se guarda el resultado para subirlo a la plataforma
write_csv(submission, "submission_nb_mice_100forest.csv")

In [15]:
options(warn = -1)
# Se calcula la probabilidad dada por el algoritmo con los datos imputados con miss forest para ambas salidas
submission$h1n1_vaccine = predict_with_ensemble_h1n1("nb", "ROC", test_feat_noNAmissF, tra_feat_noNAmissF, tra_lab, ctrl, grid)
submission$seasonal_vaccine = predict_with_ensemble_seas("nb", "ROC", test_feat_noNAmissF, tra_feat_noNAmissF, tra_lab, ctrl, grid)
options(warn = 0)

In [16]:
# Se guarda el resultado para subirlo a la plataforma
write_csv(submission, "submission_nb_missF_100forest.csv")

In [17]:
options(warn = -1)
# Se calcula la probabilidad dada por el algoritmo con los datos imputados con knn para ambas salidas
submission$h1n1_vaccine = predict_with_ensemble_h1n1("nb", "ROC", test_feat_noNAknn, tra_feat_noNAknn, tra_lab, ctrl, grid)
submission$seasonal_vaccine = predict_with_ensemble_seas("nb", "ROC", test_feat_noNAknn, tra_feat_noNAknn, tra_lab, ctrl, grid)
options(warn = 0)

In [18]:
# Se guarda el resultado para subirlo a la plataforma
write_csv(submission, "submission_nb_knn_100forest.csv")

### Ensemble OVA

#### Función OVA

In [19]:
# Se crea una función que se usa para testear el rendimiento del forest ensemble
predict_with_ova = function(submission, method, metric, tst_feat, tra_feat, tra_lab, ctrl, grid){
    # Se crea una columna de clase en la que se codifican ambas salidas
    tra_lab = tra_lab %>% mutate(clase = 2 * as.numeric(as.character(h1n1_vaccine)) + as.numeric(as.character(seasonal_vaccine)))
    # Se crea una columna de clasex para cada valor de la combinación de salidas que se pueda dar
    tra_lab = tra_lab %>% mutate(clase0 = ifelse(clase == 0, 1, 0),
                                 clase1 = ifelse(clase == 1, 1, 0),
                                 clase2 = ifelse(clase == 2, 1, 0),
                                 clase3 = ifelse(clase == 3, 1, 0))
    # Se redimensiona y se convierte a factor
    tra_lab = as.data.frame(lapply(tra_lab[, -1], as.factor))

    # Se entrena un clasificador para el caso en el que h1n1 = 0 y seasonal = 0
    Fit_c0 = train(tra_feat, make.names(tra_lab$clase0),
                    method = method,
                    metric = metric,
                    trControl = ctrl)
    # Se predice la probabilidad de que la salida multiclase sea el caso correspondiente
    Pred_c0_prob = predict(Fit_c0, newdata = tst_feat, type = "prob")

    # Se entrena un clasificador para el caso en el que h1n1 = 0 y seasonal = 1
    Fit_c1 = train(tra_feat, make.names(tra_lab$clase1),
                    method = method,
                    metric = metric,
                    trControl = ctrl)
    # Se predice la probabilidad de que la salida multiclase sea el caso correspondiente
    Pred_c1_prob = predict(Fit_c1, newdata = tst_feat, type = "prob")

    # Se entrena un clasificador para el caso en el que h1n1 = 1 y seasonal = 0
    Fit_c2 = train(tra_feat, make.names(tra_lab$clase2),
                    method = method,
                    metric = metric,
                    trControl = ctrl)
    # Se predice la probabilidad de que la salida multiclase sea el caso correspondiente
    Pred_c2_prob = predict(Fit_c2, newdata = tst_feat, type = "prob")

    # Se entrena un clasificador para el caso en el que h1n1 = 1 y seasonal = 1
    Fit_c3 = train(tra_feat, make.names(tra_lab$clase3),
                    method = method,
                    metric = metric,
                    trControl = ctrl)
    # Se predice la probabilidad de que la salida multiclase sea el caso correspondiente
    Pred_c3_prob = predict(Fit_c3, newdata = tst_feat, type = "prob")
    
    # Se hace una ponderación de las probabilidades de cada clase de las 4 multiclases para hallar la probabilidad de cada salida por separado
    submission$h1n1_vaccine = (((Pred_c2_prob$X1 + Pred_c3_prob$X1)/2 + (1 - (Pred_c0_prob$X1 + Pred_c1_prob$X1)/2))/2)
    submission$seasonal_vaccine = (((Pred_c1_prob$X1 + Pred_c3_prob$X1)/2+ (1 - (Pred_c0_prob$X1 + Pred_c2_prob$X1)/2))/2)
    
    # Se devuelve el dataset para enviarlo a la plataforma
    return(submission)
}

#### Validación con los datos de entrenamiento

In [20]:
# Se crea el control del entrenamiento y los parámetros del clasificador
ctrl <- trainControl(method="cv", number=5, classProbs= TRUE, summaryFunction = twoClassSummary)
grid <- expand.grid(.fL=c(0), .usekernel=c(FALSE),.adjust=1)

In [21]:
# Se crean subsets de los datos de entrenamiento para entrenamiento y evaluación
submission_nb_mice_ova_sub = submission[as.integer(dim(tra_feat_noNAmice)[1]*0.8):dim(tra_feat_noNAmice)[1],]
tra_feat_noNAmice_sub = tra_feat_noNAmice[1:as.integer(dim(tra_feat_noNAmice)[1]*0.8),]
tra_lab_sub = tra_lab[1:as.integer(dim(tra_feat_noNAmice)[1]*0.8),]
test_feat_noNAmice_sub = tra_feat_noNAmice[as.integer(dim(tra_feat_noNAmice)[1]*0.8):dim(tra_feat_noNAmice)[1],]
test_lab_sam = tra_lab[as.integer(dim(tra_feat_noNAmice)[1]*0.8):dim(tra_feat_noNAmice)[1],]

In [22]:
options(warn = -1)
# Se comprueba el rendimiento del clasificador en los datos de entrenamiento para comprobar el funcionamiento correcto
submission_nb_mice_ova_sub = predict_with_ova(submission_nb_mice_ova_sub, "nb", "ROC", test_feat_noNAmice_sub, tra_feat_noNAmice_sub, tra_lab_sub, ctrl, grid)
options(warn = 0)

In [23]:
cat("AUC:", auc(roc(submission_nb_mice_ova_sub$h1n1_vaccine, test_lab_sam$h1n1_vaccine)))
cat("AUC:", auc(roc(submission_nb_mice_ova_sub$seasonal_vaccine, test_lab_sam$seasonal_vaccine)))

AUC: 0.78164AUC: 0.8260442

#### Ova test

In [24]:
submission_nb_missF_ova = submission
options(warn = -1)
# Se ejecuta el ensemble OVA y se guardan las predicciones de las probabilidades en el formato correspondiente para los datos imputados con mice
submission_nb_mice_ova = predict_with_ova(submission_nb_missF_ova, "nb", "ROC", test_feat_noNAmice, tra_feat_noNAmice, tra_lab, ctrl, grid)
options(warn = 0)

In [25]:
# Se guarda el resultado para subirlo a la plataforma
write_csv(submission_nb_mice_ova, "submission_nb_mice_ova.csv")

In [26]:
submission_nb_missF_ova = submission
options(warn = -1)
# Se ejecuta el ensemble OVA y se guardan las predicciones de las probabilidades en el formato correspondiente para los datos imputados con miss forest
submission_nb_missF_ova = predict_with_ova(submission_nb_missF_ova, "nb", "ROC", test_feat_noNAmissF, tra_feat_noNAmissF, tra_lab, ctrl, grid)
options(warn = 0)

In [27]:
# Se guarda el resultado para subirlo a la plataforma
write_csv(submission_nb_missF_ova, "submission_nb_missF_ova.csv")

In [28]:
submission_nb_knn_ova = submission
options(warn = -1)
# Se ejecuta el ensemble OVA y se guardan las predicciones de las probabilidades en el formato correspondiente para los datos imputados con knn
submission_nb_knn_ova = predict_with_ova(submission_nb_knn_ova, "nb", "ROC", test_feat_noNAknn, tra_feat_noNAknn, tra_lab, ctrl, grid)
options(warn = 0)

In [29]:
# Se guarda el resultado para subirlo a la plataforma
write_csv(submission_nb_knn_ova, "submission_nb_knn_ova.csv")

### Ensemble OVO + Boosting

In [30]:
boosting_nb_model = function(train, test, n.weak.learners = 100, probability.calibration = "friedman") {
  
  # Declaramos las variables necesarias para ejecutar el boosting
  models.trained = 0
  
  # Definimos los pesos iniciales para nuestro conjunto de datos de entrenamiento
  weigths = rep(c(1/dim(train)[1]), dim(train)[1])
    
  # Creamos un vector para almacenar las importancias de los clasificadores débiles
  weak.learners.importance = c()
  
  # Creamos una lista en la que almacenar 
  weak.learners.test.predictions = list()
  
  # Iteramos mientras queden modelos que entrenar
  while(models.trained < n.weak.learners){
    
    print(paste("[Boosting Model] Entrenando clasificador débil número ", models.trained+1, "...", sep = ""))
    
    # En el caso de que sea el primer modelo utilizamos como datos el conjunto de entrenamientp
    if(models.trained == 0){
      sample.indexes = 1:nrow(train)
      iteration.data = train
    }else{
      # En el caso contrario, realizamos una muestra ponderada del conjunto de datos de
      # entrenamiento utilizando los pesos calculados
      sample.indexes = sample(seq_len(nrow(train)), nrow(train), prob=weigths)
      iteration.data = train[sample.indexes, ]
    }
      
    
    # Entrenamos el modelo sobre el conjunto de datos de entrenamiento
    Fit_seas_i = train(iteration.data[-dim(iteration.data)[1]], make.names(iteration.data$class),
              method = "nb")

    #model.Ripper = JRip(class~., iteration.data)
    
    # Realizamos predicciones sobre el propio conjunto de datos
    #model.Ripper.pred = predict(model.Ripper, newdata = iteration.data)
      
    model.Ripper.pred = predict(Fit_seas_i, newdata = iteration.data)
    
    # Calculamos el error del clasificador débil
    training.error = 0
    for(training.index in 1:length(model.Ripper.pred)){
      
      # Obtenemos el índice de esta observación al realizar el muestreo
      sample.idx = sample.indexes[training.index]
        
      # Calculamos el error para esta observación de la muestra y la añadimos a nuestros datos
      training.error = training.error + weigths[sample.idx]*as.integer(model.Ripper.pred[training.index] == make.names(train[sample.idx, "class"]))
    }
    training.error = training.error / sum(weigths)
    
    # Calculamos el alpha para actualizar los pesos
    alpha = log((1 - training.error) / training.error)
    
    # Añadimos la importancia de este clasificador a la estructura de datos designada para esta labor
    weak.learners.importance = c(weak.learners.importance, alpha)

    # Actualizamos los pesos de las observaciones
    for(training.index in 1:length(model.Ripper.pred)){
      
      # Obtenemos el índice de esta observación al realizar el muestreo
      sample.idx = sample.indexes[training.index]
      
      # Calculamos el nuevo peso para esta observación del conjunto de datos
      weight.computed = weigths[sample.idx]*exp(alpha*as.integer(model.Ripper.pred[training.index] == make.names(train[sample.idx, "class"])))
        
      # Modificamos el peso en el vector original
      weigths[sample.idx] = weight.computed
    }
      
    
    
    print(paste("[Boosting Model] Realizando predicciones con el clasificador débil número ", models.trained+1, "...", sep = ""))
    
    # Realizamos con este clasificador débil predicciones sobre el conjunto de datos de evaluación
    model.Ripper.test.pred = predict(Fit_seas_i, newdata = iteration.data, type = "prob")
    
    # Añadimos estas predicciones a la lista de resultados
    weak.learners.test.predictions[[models.trained + 1]] = model.Ripper.test.pred
    
    print(paste("[Boosting Model] Entrenamiento y evaluación del clasificador débil número ", models.trained+1, " completado.", sep = ""))
  
    # Añadimos una unidad a los modelos ya entrenados
    models.trained = models.trained + 1
  }
  
  print("[Boosting Model] Agregando la salida de los modelos...")
  
    
  # Calculamos la salida del modelo para cada observación del conjunto de datos de evaluación
  final.output.boosting = c()
  for(test.observation.index in 1:(nrow(test)/2)) {
    
    # Para cada uno de los clasificadores, multiplicamos su salida por la importancia del clasificador
    # output.boosting = sapply(1:n.weak.learners, function(weak.model.index) 
    #  weak.learners.importance[weak.model.index] * ifelse(weak.learners.test.predictions[[weak.model.index]][test.observation.index] == 1, 1, -1))
    # Añadimos el output final de este modelo a un vector
    #final.output.boosting = c(final.output.boosting, sum(unlist(output.boosting)))

    # Obtenemos las predicciones de los diferentes clasificadores para esta observación
    predictions.for.observation = unlist(lapply(weak.learners.test.predictions, function(predictions) predictions[test.observation.index,]))
    # Calculamos la probabilidad haciendo uso del paper de Friedman
    fx = 0.5 * log((sum(predictions.for.observation[2]>predictions.for.observation[1]) + sum(predictions.for.observation[4]>predictions.for.observation[3]))/length(predictions.for.observation)
                   / (sum(predictions.for.observation[1]>predictions.for.observation[2]) + (sum(predictions.for.observation[3]>predictions.for.observation[4])))/length(predictions.for.observation))
    boosting.prob = 1 / (1 + exp(-2*fx))
    
    # Añadimos la predicción de probabilidad para esta observación en nuestra estructura de datos
    final.output.boosting = c(final.output.boosting, boosting.prob)
                                                
    # Calculamos la probabilidad haciendo uso del paper de Friedman
    fx = 0.5 * log((sum(predictions.for.observation[4]>predictions.for.observation[3]))/length(predictions.for.observation)
                   / ((sum(predictions.for.observation[3]>predictions.for.observation[4])))/length(predictions.for.observation))
    boosting.prob = 1 / (1 + exp(-2*fx))
    
    # Añadimos la predicción de probabilidad para esta observación en nuestra estructura de datos
    final.output.boosting = c(final.output.boosting, boosting.prob)
  }
  
  print("[Boosting Model] Modelo de Boosting instanciado y evaluado con éxito.")
                                     
  final.output.boosting
}

In [6]:
X_train = tra_feat_noNAknn
X_test = test_feat_noNAknn

options(warn = -1)

# Cargamos el formato de submission y las etiquetas correspondientes a las clases del conjunto de entrenamiento
y_train = read.csv("training_set_labels.csv", header=TRUE, sep=',', na.strings = c('?','', 'NA'))
submission_format = read.csv("submission_format.csv", header=TRUE, sep=',', na.strings = c('?','', 'NA'))

# Modificamos las clases para crear una única variable multiclase y la añadimos a nuestros datos de entrenamiento
X_train['class'] = strtoi(paste(y_train$h1n1_vaccine, y_train$seasonal_vaccine, sep=""), base = 2)

# Convertimos esta clase en un factor
X_train = X_train %>%
  mutate(class = factor(class, levels = c(0, 1, 2, 3), labels = c("Unvaccinated", "Vaccinated (Seasonal)", "Vaccinated (H1N1)", "Vaccinated (Both)")))

# Obtenemos todas las parejas posibles de clases que existen
class_pairs = list()
class_pairs[[1]] = c(levels(X_train$class)[1], levels(X_train$class)[2])
class_pairs[[2]] = c(levels(X_train$class)[1], levels(X_train$class)[3])
class_pairs[[3]] = c(levels(X_train$class)[1], levels(X_train$class)[4])
class_pairs[[4]] = c(levels(X_train$class)[2], levels(X_train$class)[3])
class_pairs[[5]] = c(levels(X_train$class)[2], levels(X_train$class)[4])
class_pairs[[6]] = c(levels(X_train$class)[3], levels(X_train$class)[4])

# Creamos una función que dado lo siguiente:
#  - Un conjunto de datos de entrenamiento con una columna 'class' con las clases a predecir
#  - El conjunto de datos de evaluación a predecir
#  - Un vector de etiquetas con las que filtrar los datos para entrenar el modelo
# Devuelve un vector con la probabilidad de la primera clase a predecir
calculate_probability_ovo_boosting_model = function(train_data, test_data, filtered_classes, n.weak.learners = 256) {
  
  # Filtramos el conjunto de datos por las librerias especificadas
  X_train_filtered = train_data %>%
    filter(class %in% filtered_classes)
  
  # Redefinimos el factor para no generar problemas con Caret
  X_train_filtered = X_train_filtered %>%
    mutate(class = factor(class, levels = filtered_classes))
  
  # Transformamos la clase a 0 y 1
  X_train_filtered = X_train_filtered %>%
    mutate(class = ifelse(class == filtered_classes[1], 1, 0))
  
  # Lo volvemos a convertir en un factor
  X_train_filtered = X_train_filtered %>%
    mutate(class = as.factor(class))
  
  # Creamos el modelo de boosting y evaluamos sobre el conjunto de evaluación
  boosting.result = boosting_nb_model(X_train_filtered, test_data, n.weak.learners = n.weak.learners)
  boosting.result
}

# Lanzamos este método para cada una de las parejas de clases que hemos planteado
predicted.ovo = lapply(class_pairs, function(pair) calculate_probability_ovo_boosting_model(X_train, X_test, unlist(pair), n.weak.learners = 2))
predicted.ovo

# Definimos el número de clases
K = 4

# Declaramos un vector de etiquetas con los nombres de las probabilidades obtenidas
probabilities_names = unlist(lapply(class_pairs, function(pair) paste(pair[1], pair[2], sep=" Vs. ")))

# Creamos dos vectores para almacenar los resultados para cada una de las vacunas
h1n1.probabilities = c()
seas.probabilities = c()

# Iteramos sobre cada uno de los elementos de las predicciones devueltas por los modelos OVO
for(prediction.index in 1:length(predicted.ovo[[1]])){
  
  # Obtenemos un array con las predicciones para la instancia en concreto
  probabilities_vector = unlist(lapply(predicted.ovo, function(predictions) predictions[prediction.index]))
  probabilities_array = array(probabilities_vector, dim=c(1,(K*(K-1)/2)))
  colnames(probabilities_array) = probabilities_names
  
  # Obtenemos la matriz con las probabilidades
  Q = matrix(0,K,K)
  Q[lower.tri(Q)] = 1 - probabilities_array
  Qt = t(Q)
  Q[upper.tri(Q)] = 1 - Qt[upper.tri(Qt)]
  diag(Q) = rowSums(Q)
  Q = Q / (K-1)
  
  # Creamos un vector de probabilidades para cada clase inicial que sume 1
  p = rbeta(K,1,1)
  p = p/sum(p)
  
  # Actualizamos el vector de probabilidades hasta que el equilibrio ha sido alcanzado utilizando
  # como referencia: Probability Estimates for Multi-class Classification by Pairwise Coupling, Wu et al. (2003)
  for(i in 1:1000) p = Q%*%p
                                       
  
  # Formateamos un poco el vector de probabilidades resultante
  final.probability = as.vector(t(p))
  names(final.probability) = levels(X_train$class)
  
  # Calculamos la probabilidad de que haya tomado ambas vacunas siguiendo el formato establecido
  h1n1.vaccine.prob = final.probability['Vaccinated (H1N1)'] + final.probability['Vaccinated (Both)']
  seas.vaccine.prob = final.probability['Vaccinated (Seasonal)'] + final.probability['Vaccinated (Both)']
  
  # Añadimos estas probabilidades a los vectores en los que las almacenamos
  h1n1.probabilities = append(h1n1.probabilities, h1n1.vaccine.prob)
  seas.probabilities = append(seas.probabilities, seas.vaccine.prob)
}

# Añadimos al submission format las predicciones de probabilidad 
submission_format['h1n1_vaccine'] = h1n1.probabilities
submission_format['seasonal_vaccine'] = seas.probabilities
submission_format

options(warn = -1)
                                       
# Guardamos en disco los resultados
write.csv(submission_format,"submission_nb_knn_ovo_boosting.csv", row.names = FALSE)

In [31]:
submission_format[11348,]

,respondent_id,h1n1_vaccine,seasonal_vaccine
,<int>,<dbl>,<dbl>
11348,38054,0,7.188294e-177


In [30]:
submission_format[11349,]

,respondent_id,h1n1_vaccine,seasonal_vaccine
,<int>,<dbl>,<dbl>
11349,38055,NA,NA


A partir del 11349 son todo NAs